In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [2]:
def search(q):
    q = q.replace('\n', '').replace('（', '(').replace('）', ')')
    print("START : " + q)
    q = re.sub("\(.+\)$", "", " ".join(q.split()[:-3]), re.UNICODE)
    query = re.sub('(!|\u3000|/|\s|>|<|\.)+', "%20", q)
    url_search = 'https://eiga.com/search/' + query
    res_search = requests.get(url_search )
    res_search.encoding = res_search.apparent_encoding
    soup_search = BeautifulSoup(res_search.content, "lxml")
    result =  soup_search.find('section', attrs={"id": "rslt-movie"})
    if(result != None):
        path = result.find('li', attrs={"class": "col-s-3"}).find('a')["href"]
        url_review = 'https://eiga.com' + path
        return url_review
    else:
        print("**************************************************")
        print(q + " HAS NO RESULT")
        print("**************************************************")
        return "error"

In [3]:
def scrape(query):
    page_num = 1
    data = {
        "id": -1,
        "rating": -1,
        "check-in":-1,
        "review-count":-1
    }
    url_review=search(query)
    
    if(url_review == "error"):
        return "error"
    

    res = requests.get(url_review)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content, "lxml")
    
    rating = soup.find('span', attrs={"class": "rating-star"})
    review_count = soup.find('span', attrs={"itemprop": "reviewCount"})
    check_in = soup.find('a', attrs={"class": "icon-movie-checkin"}).find('strong')
    data["rating"] = 0 if rating.text == '－' else float(rating.text)
    data["review-count"] = 0 if review_count is None else int(review_count.text)
    data["check-in"] = 0 if check_in is None else int(check_in.text)
        
    return data

In [4]:
input_file = '../2018_movie_clean'

In [5]:
num = 1
for q in open(input_file, 'r', encoding='utf-8').readlines():
    output_file = './{0}.json'.format(num)
    if(num <= 700):
        num += 1
        continue
    
    print(num)
    with open(output_file, 'w') as f:
        data = scrape(q)
        if(data == "error"):
            num += 1
            continue
        data["id"] = num
        jsn =  json.dumps(data,ensure_ascii=False, indent=2) 
        f.write(jsn)   
    f.close()
    num += 1
    time.sleep(1)

701
START : 第三世代	2018	10	27
702
START : ノーマーク爆牌党	2018	10	27
703
START : 映画 HUGっと!プリキュア・ふたりはプリキュア オールスターズメモリーズ	2018	10	27
704
START : 宮古島からのSOS	2018	10	27
705
START : 殺る女	2018	10	27
706
START : 夜明けまで離さない	2018	10	27
707
START : ライ麦畑で出会ったら	2018	10	27
708
START : わたしの居場所 新世界物語	2018	10	27
709
START : 居酒屋ばぁば	2018	11	1
**************************************************
居酒屋ばぁば HAS NO RESULT
**************************************************
710
START : ビブリア古書堂の事件手帖	2018	11	1
711
START : 赤毛のアン 卒業	2018	11	2
712
START : ヴェノム	2018	11	2
713
START : 宇宙戦艦ヤマト2202 愛の戦士たち/第六章 回生篇	2018	11	2
714
START : 華氏119	2018	11	2
715
START : シャークネード ラスト・チェーンソー 4DX	2018	11	2
716
START : スマホを落としただけなのに	2018	11	2
717
START : METライブビューイング2018-19/ヴェルディ《アイーダ》	2018	11	2
718
START : 狂獣 欲望の海域	2018	11	3
719
START : 熊川哲也 Kバレエ カンパニー 「コッペリア」 in Cinema	2018	11	3
720
START : K SEVEN STORIES Episode 5 「メモリー・オブ・レッド ～BURN～」	2018	11	3
721
START : The Witch/魔女	2018	11	3
722
START : 十年 Ten Years Japan	2018	11	3
723
START : シンプル・ギフト〜はじまりの

892
START : 小河ドラマ 龍馬がくる	2018	12	15
893
START : スモーキング・エイリアンズ	2018	12	15
894
START : ナニワノノア	2018	12	15
895
START : 葡萄畑に帰ろう (ジョージア (国)の旗 ジョージア)	2018	12	15
896
START : ほっぷすてっぷじゃんぷッ！	2018	12	15
897
START : マイ・サンシャイン	2018	12	15
898
START : メアリーの総て	2018	12	15
899
START : モーターヘッド/クリーン・ユア・クロック	2018	12	15
900
START : 夢こそは、あなたの生きる未来	2018	12	15
901
START : 宵闇真珠	2018	12	15
902
START : レッド・ブレイド	2018	12	15
903
START : 輪違屋糸里 京女たちの幕末	2018	12	15
904
START : ほっぷすてっぷじゃんぷッ！2	2018	12	16
905
START : アリー/ スター誕生	2018	12	21
906
START : シュガー・ラッシュ:オンライン	2018	12	21
907
START : ニセコイ	2018	12	21
908
START : 私は、マリア・カラス	2018	12	21
909
START : いつか家族に	2018	12	22
910
START : 家へ帰ろう	2018	12	22
911
START : 平成仮面ライダー20作記念 仮面ライダー平成ジェネレーションズ FOREVER	2018	12	22
912
START : からっぽ	2018	12	22
913
START : クマ・エロヒーム	2018	12	22
914
START : サイドマン:スターを輝かせた男たち	2018	12	22
915
START : シシリアン・ゴースト・ストーリー	2018	12	22
916
START : 死体が消えた夜	2018	12	22
917
START : 声優ボウリングランプリ2	2018	12	22
918
START : バスキア、10代最後のとき	2018	12	22
919
START : プラネット・オブ・アメーバ	2